In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

In [ ]:
import { RecursiveSet, Tuple } from 'recursive-set';
type Variable = string;
type Literal  = Variable | Tuple<['¬', Variable]>;
type Clause   = RecursiveSet<Literal>;
type Clauses   = RecursiveSet<Clause>;

The <a href="https://en.wikipedia.org/wiki/Eight_queens_puzzle">eight queens puzzle</a> is the problem of placing eight chess queens on a chessboard so that no two queens can attack each other.  In <a href="https://en.wikipedia.org/wiki/Chess">chess</a> a queen can attack another piece if this piece is either
<ol>
    <li>in the same row,</li>
    <li>in the same column, or</li>
    <li>in the same diagonal.</li>
</ol>
The image below shows a queen in row 3, column 4.  All the locations where a piece can be captured by this queen are marked with an arrow.

<img src="queen-captures.png">

We will solve this puzzle by coding it as a formula of propositional logic.  This formula will be solvable iff the eight queens puzzle has a solution.  We will use the algorithm of *Davis and Putnam* to compute the solution of this formula.

In [ ]:
import * as DP from './07-Davis-Putnam-JW';

The function $\texttt{var}(r, c)$ takes a row $r$ and a column $c$ and returns the string $\texttt{'Q<}r\texttt{,}c\texttt{>'}$.  This string is interpreted as a propositional variable specifying that there is a queen in row $r$ and column $c$.  The image below shows how theses variables correspond to the positions on a chess board.

<img src="queens-vars.png">

The function `varName(row, col)` takes two integers `row` and `col` as its argument and returns a string of the form `f'Q<{row},{col}>`.
This string is interpreted as a propositional variable.  This variable is `true` if there is a queen in the given row and column on the board.

In [ ]:
function varName(row: number, col: number): Variable {
    return `Q<${row},${col}>`;
}

In [ ]:
varName(11,3);

Given a set of propositional variables $S$, the function $\texttt{atMostOne}(S)$ returns a set containing clauses that expresses the fact that **at most one** of the variables in $S$ is `True`.

In [ ]:
function atMostOne(S: RecursiveSet<Variable>): RecursiveSet<Clause> {
    const result = new RecursiveSet<Clause>();
    const arr: Variable[] = [];
    for(const v of S) {
        arr.push(v as Variable);
    }
    for (let i = 0; i < arr.length; i++) {
        for (let j = i + 1; j < arr.length; j++) {
            const p = arr[i];
            const q = arr[j];

            const clause = new RecursiveSet<Literal>();
            clause.add(new Tuple('¬', p));
            clause.add(new Tuple('¬', q));

            result.add(clause);
        }
    }
    return result;
}

In [ ]:
const vars = new RecursiveSet<Variable>('a', 'b', 'c');
const clauses = atMostOne(vars);
console.log(clauses.toString());

Given a `row` and the size of the board `n`, the procedure `atMostOneInRow(row, n)` computes a set of clauses that is `True` if and only there is at most one queen in the
given row.

In [ ]:
function atMostOneInRow(row: number, n: number): RecursiveSet<Clause> {
    const VarsInRow = new RecursiveSet<Variable>();
    for (let col = 1; col <= n; col++) {
        VarsInRow.add(varName(row, col));
    }
    return atMostOne(VarsInRow);
}

In [ ]:
atMostOneInRow(3, 4);

Given a column `col` and the size of the board `n`, the procedure `oneInColumn(col, n)` computes a set of clauses that is true if and only if there is at least one queen in the given column.

In [ ]:
function oneInColumn(col: number, n: number): RecursiveSet<Clause> {
    const VarsInColumn = new RecursiveSet<Literal>();
    for (let row = 1; row <= n; row++) {
        VarsInColumn.add(varName(row, col));
    }
    const result = new RecursiveSet<Clause>();
    result.add(VarsInColumn as Clause);
    return result;
}

In [ ]:
oneInColumn(2, 4);

Given a number `k` and the size of the board `n`, the procedure `atMostOneInFallingDiagonal(k, n)` computes a set of clauses that is `True` if and only if there 
is at most one queen in the falling diagonal specified by the equation
```
     row - col = k.
```

In [ ]:
function atMostOneInFallingDiagonal(k: number, n: number): RecursiveSet<Clause> {
    const VarsInDiagonal = new RecursiveSet<Variable>();
    for (let row = 1; row <= n; row++) {
        for (let col = 1; col <= n; col++) {
            if (row - col === k) {
                VarsInDiagonal.add(varName(row, col));
            }
        }
    }
    return atMostOne(VarsInDiagonal);
}

In [ ]:
atMostOneInFallingDiagonal(0, 4);

Given a number `k` and the size of the board `n`, the procedure `atMostOneInRisingDiagonal(k, n)` computes a set of clauses that is `True` 
if and only if there is at most one queen in the rising diagonal specified by the equation
```
    row + col = k. 
```

In [ ]:
function atMostOneInRisingDiagonal(k: number, n: number): RecursiveSet<Clause> {
    const VarsInDiagonal = new RecursiveSet<Variable>();
    for (let row = 1; row <= n; row++) {
        for (let col = 1; col <= n; col++) {
            if (row + col === k) {
                VarsInDiagonal.add(varName(row, col));
            }
        }
    }
    return atMostOne(VarsInDiagonal);
}

In [ ]:
atMostOneInRisingDiagonal(5, 4);

The function `allClauses(n)` takes the size of the board $n$ and computes a set of clauses that specify that
* there is at most one queen in every row,
* there is at most one queen in every rising diagonal,
* there is at most one queen in every falling diagonal, and
* there is at least one queen in every column.

In [ ]:
function allClauses(n: number): RecursiveSet<Clause> {
    const all: Array<RecursiveSet<Clause>> = [];
    for (let row = 1; row <= n; row++) {
        all.push(atMostOneInRow(row, n));
    }
    for (let k = 3; k <= 2 * n; k++) {
        all.push(atMostOneInRisingDiagonal(k, n));
    }
    for (let k = -(n - 2); k <= n - 2; k++) {
        all.push(atMostOneInFallingDiagonal(k, n));
    }
    for (let col = 1; col <= n; col++) {
        all.push(oneInColumn(col, n));
    }
    const result = new RecursiveSet<Clause>();
    for (const clauses of all) {
        for (const clause of clauses) {
             result.add(clause);
        }
    }
    return result;
}

In [ ]:
const clauses = allClauses(16);
for (const clause of clauses) {
    console.log(clause.toString());
}

The set of all clauses contains 512 clauses.  There are 64 variables.  

In [ ]:
console.log(clauses.size);

The function $\texttt{queens}(n)$ solves the *n queens problem*.

In [ ]:
function queens(n: number): RecursiveSet<Clause> {
    // "Solve the n queens problem."
    const Clauses = allClauses(n);
    const Solution = DP.solve(Clauses);
    const EmptyClause = new RecursiveSet<Literal>();
    if (Solution.has(EmptyClause)) {
        console.log(`The problem is not solvable for ${n} queens!`);
    }
    return Solution;
}

The *8 queens problem* can be solved in less than a tenth of a second using the 
pure *Davis Putnam algorithm* that does not use the Jereslow-Wang heuristic. If we want to solve the *16 queens problem*, we need to use the Jereslow-Wang heuristic.  With this heuristic, the *16 queens problem* is then solved in roughly 4 seconds on my Mac Studio from 2023.

In [ ]:
console.time('queens');
const Solution: RecursiveSet<Clause> = queens(16);
console.timeEnd('queens');
if (Solution) for (const C of Solution) console.log(`${C}`);

# Visualization

The function `removeNegativeLiterals` takes one input:
* `Solution` is a set of unit clauses.

The function returns the set of all those unit clauses in `Solution` that do not contain negative literals.

In [ ]:
function removeNegativeLiterals(Solution: Clauses): RecursiveSet<Variable> {
    const Result = new RecursiveSet<Variable>();
    for (const clause of Solution) {
        for (const lit of clause) {
            if (typeof lit === 'string') {
                Result.add(lit);
            }
        }
    }
    return Result;
}

In [ ]:
const inputSet = new RecursiveSet<Clause>(
    new RecursiveSet<Literal>('p'),
    new RecursiveSet<Literal>(new Tuple('¬', 'q')),
    new RecursiveSet<Literal>(new Tuple('¬', 'r')),
    new RecursiveSet<Literal>('s')
);
console.log(removeNegativeLiterals(inputSet).toString());

The function `extractRowCol` takes one argument:
* `varName` is a string of the form `f'Q<{row},{col}>'`.

It returns the pair `row, col`.

In [ ]:
function extractRowCol(varName: string): [string, string] {
    const left = varName.indexOf('<');
    const comma = varName.indexOf(',');
    const right = varName.indexOf('>');
    const row = varName.substring(left + 1, comma);
    const col = varName.substring(comma + 1, right);
    return [row, col];
}

In [ ]:
console.log(extractRowCol('Q<13,9>'));

In [ ]:
function transform(Solution: RecursiveSet<Clause>): Record<number, number> {
    const positiveLiterals = removeNegativeLiterals(Solution);
    const Result: Record<number, number> = {};
    for (const name of positiveLiterals) {
        const [row, col] = extractRowCol(name as string);
        Result[parseInt(row, 10)] = parseInt(col, 10);
    }
    return Result;
}

The function `showSolution(Solution, width)` takes a dictionary that contains a variable assignment that represents a solution to the n queens puzzle. It displays this Solution on a chess board.

* `Solution`: A `RecursiveSet` containing the variable assignment (as unit clauses).  
  The function transforms this set into a dictionary mapping row indices to column indices.
  If `transformed[row] = col`, then the queen in `row` is placed in `col`.
* `width`: Specifies the size of the board as a percentage of the width of notebook.

In [ ]:
import { display } from 'tslab';

function showSolution(Solution: RecursiveSet<Clause>, width = "50%") {
    const transformed = transform(Solution);
    const n = Object.keys(transformed).length;
    const boardArray: string[][] = Array.from({ length: n }, () => Array(n).fill(''));
    
    for (let row = 1; row <= n; row++) {
        const col = transformed[row];
        if (col !== undefined) {
            if (row - 1 < n && col - 1 < n) {
                boardArray[row - 1][col - 1] = '♕';
            }
        }
    }
    let html = `<div style="display:grid; grid-template-columns:repeat(${n}, 1fr); width:${width}; aspect-ratio: 1/1; border: 2px solid black;">`;
    for (let row = 0; row < n; row++) {
        for (let col = 0; col < n; col++) {
            const piece = boardArray[row][col];
            const bgColor = (row + col) % 2 === 0 ? '#f0d9b5' : '#b58863'; 
            
            let cellContent = '';
            if (piece) {
                cellContent = `
                <svg viewBox="0 0 100 100" style="width: 80%; height: 80%; display: block;">
                    <text x="50%" y="55%" font-size="90" text-anchor="middle" dominant-baseline="middle" fill="black">
                        ${piece}
                    </text>
                </svg>`;
            }
            html += `<div style="
                display: flex; 
                align-items: center; 
                justify-content: center; 
                background-color:${bgColor};
                overflow: hidden;
                ">${cellContent}</div>`;
        }
    }
    html += `</div>`;
    display.html(html);
}

In [ ]:
for (const C of Solution) console.log(`${C}`);

In [ ]:
showSolution(Solution, "50%");